In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
# 全部的PCODE表（CAP提供）
pcode = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\產品規格相關\BPCOM\BPCOM_20230103.xlsx")
pcode = pcode[["PrCode", "GRP", "Section", "Rim", "BP", "Product Name"]]

In [3]:
# 全部的Premium list
premium = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\KPI\2023 GKPI\20230710_GKPI BSFC_June 2023.xlsx", sheet_name="Premium Products - BPCs")
premium = premium.iloc[1:]
premium = premium[["BPC", "Premium"]]

In [4]:
# 產品總表
pro_df = pd.merge(pcode, premium, how="left", left_on="PrCode", right_on="BPC").drop("BPC", axis=1)

In [72]:
# 整理實績和下半年RT3
# 實績整理

act = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\20230710_SizeData.xlsx", sheet_name="Sheet1")

In [73]:
act = act[(act["預算/前實績"] == "23ACT") & (act["通路明細"] != "LSCM") & (act["客戶名稱"] != "普利司通")]

In [74]:
act.groupby(["通路明細"])[["條數"]].sum()

,條數
通路明細,
AG,186
CD,515
FLT-BSSC,6502
FLT-REP,1598
FT,4857
MM,0
OT,2
中盤,988
加油站,10


In [75]:
channel_keys = ['FLT-REP', 'CD', 'FLT-BSSC', 'OT', '中盤', '加油站', 'AG', '輪胎行', 'FT', 'MM', 'BSSC', 'BSSC-FLT']
channel_values = ['FLT-REP', 'T/S', 'BSSC-FLT', 'T/S', 'T/S', 'T/S', 'T/S', 'T/S', 'FLT-REP', 'T/S', 'T/S', 'BSSC-FLT']
channel_dict = dict(zip(channel_keys, channel_values))

In [76]:
act["通路明細"] = act["通路明細"].map(channel_dict)

In [45]:
act_result = act.groupby([pd.Grouper(key="整理後的日期", freq="MS"), "商品代號", "通路明細"])[["條數"]].sum().reset_index()

In [46]:
act_result["整理後的日期"] = act_result["整理後的日期"].dt.month_name()

In [109]:
## RBC3整理
rba = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\價格相關\預算\2023\20230801_RBAC3_final.xlsx", sheet_name="工作表1")

In [110]:
rba["整理後的日期"] = pd.to_datetime(rba["年月"], format="%Y%m")

In [111]:
rba = rba[(rba["國內市場財別"] == "TBR(G)") & (rba["預算/前實績"] == "23RBAC3") & (rba["大通路分類"] != "OE") & (rba["通路明細"] != "LSCM") & (rba["客戶名稱"] != "普利司通") & (rba["整理後的日期"] >= "2023-07-01")]

In [112]:
rba["通路明細"] = rba["通路明細"].map(lambda x: "T/S" if x in ["AG", "BSSC"] else x)

In [65]:
rba_result = rba.groupby([pd.Grouper(key="整理後的日期", freq="MS"), "商品代號", "通路明細"])[["條數"]].sum().reset_index()

In [66]:
rba_result["條數"] = rba_result["條數"].astype(int)

In [67]:
rba_result["整理後的日期"] = rba_result["整理後的日期"].dt.month_name()

In [68]:
## 整理業務填回的數據
ob = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\OB\24OB\24OB_業務填寫表格.xlsx", None)

In [69]:
dfs = []
columns = ["PCODE",	"中計", "上市時間",	"GRP", "大通路", "客戶通路", "規格", "吋別", "花紋", "產地", "Product Name", "BSCAP definition", "1月",	"2月", "3月", "4月", "5月",	"6月", "7月", "8月", "9月",	"10月",	"11月",	"12月",	"2025年", "2026年"]
for sheet in ob.keys():
    if sheet not in ['歷史數據樞紐', 'Raw data', 'Demand_Sales', 'TBR premium定義', '總表', '24OB表格']:
        df = ob[sheet].iloc[8:]
        df.columns = columns
        # df["業務名稱"] = sheet
        dfs.append(df)

In [70]:
ob_result = pd.concat(dfs)

In [71]:
ob_result = ob_result[['PCODE', '客戶通路', '1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月', '2025年', '2026年']]

In [72]:
ob_result["客戶通路"] = ob_result["客戶通路"].map(lambda x: "T/S" if x in ["FMC", "其他"] else x)

In [73]:
ob_result = ob_result.groupby(['PCODE', '客戶通路']).sum().stack().reset_index(name="條數")

In [74]:
ob_result.columns = ["商品代號", "通路明細", "整理後的日期", "條數"]

In [75]:
ob_result = ob_result[["整理後的日期", "商品代號", "通路明細", "條數"]]

In [76]:
# 合併表格
final = pd.concat([act_result, rba_result, ob_result], ignore_index=True)

In [77]:
final = final.set_index(["商品代號", "通路明細", "整理後的日期"]).unstack("整理後的日期", fill_value=0).droplevel(level=0, axis=1).reset_index()

In [78]:
final = final.rename_axis(None, axis=1)

In [79]:
final = final[['商品代號', '通路明細', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', '1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月', '2025年', '2026年']]

In [80]:
## 讀取產品總表
pro_table = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\OB\24OB\24OB_product_table_updated.csv")

In [81]:
def rim_transform(df):
    if df == "16C" or len(df) == 2:
        return df
    else:
        return df[:2] + "." + df[-1]

In [82]:
pro_table["Rim"] = pro_table["Rim"].map(rim_transform)

In [83]:
pro_table["size"] = pro_table["Section"] + "R" + pro_table["Rim"]

In [84]:
pcode_dict = dict(zip(pro_table["PrCode"], pro_table[["size", "BP"]].apply(tuple, axis=1)))

In [85]:
final["產品規格"] = final["商品代號"].map(pcode_dict)

In [86]:
final[["Size", "Pattern"]] = pd.DataFrame(final["產品規格"].tolist(), columns=["Size", "Pattern"])

In [87]:
final = final[['商品代號', '通路明細', 'Size', 'Pattern', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December', '1月',
       '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月',
       '2025年', '2026年']]

In [88]:
final.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\OB\24OB\20230803_24OB第三版數量總表格.xlsx", index=False)

In [192]:
pro_dict = dict(zip(pro_table["PrCode"], pro_table["GRP"]))

In [195]:
final["分類"] = final["商品代號"].map(pro_dict)

In [204]:
f_w = final[['分類', '1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月',
       '9月', '10月', '11月', '12月', '2025年', '2026年']]

In [212]:
f_w.set_index(["分類"]).stack().reset_index(name="數量").rename(columns={"level_1": "年月"}).to_clipboard(index=False)